In [32]:
import cv2
import pickle
import numpy as np

# ==============================
# Configuration (adjust as needed)
# ==============================
video_path = '/nas/Chingiz/sing_language/code/imgTOvideo/imgTOvideo/01April_2010_Thursday_heute_default-2/20_1/original.mp4'
output_video_path = r'/nas/Chingiz/sing_language/code/imgTOvideo/imgTOvideo/01April_2010_Thursday_heute_default-2/20_1/hand_points_overlay.mp4'
pkl_path   = r'/nas/Dataset/Phoenix/Chingiz/phoenix-2014-hands-keypoints.pkl'

# Set the video key that matches the one in your pickle file.
video_key = "fullFrame-210x260px/train/01April_2010_Thursday_heute_default-2/1/01April_2010_Thursday_heute"

# Confidence threshold: only draw keypoints with confidence above this value.
CONF_THRESH = 0.5

# ==============================
# Load hand keypoints from pickle
# ==============================
with open(pkl_path, 'rb') as f:
    hand_data = pickle.load(f)

if video_key not in hand_data:
    raise ValueError(f"Video key '{video_key}' not found in the pickle file.")

# Expected shape: (n_frames, 42, 3)
hand_points = hand_data[video_key]['keypoints']

# ==============================
# Open video file and set up VideoWriter
# ==============================
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Error opening video file!")

frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps          = cap.get(cv2.CAP_PROP_FPS)
frame_count  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video size: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {frame_count}")
print(f"Keypoints available for {hand_points.shape[0]} frames.")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# ==============================
# Process video frame by frame
# ==============================
frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Check if we have keypoints for the current frame.
    if frame_idx < hand_points.shape[0]:
        kp_frame = hand_points[frame_idx]  # shape: (42, 3)
        for point in kp_frame:
            x, y, conf = point
            if conf >= CONF_THRESH:
                cv2.circle(frame, (int(x), int(y)), radius=3, color=(0, 255, 0), thickness=-1)
    
    out.write(frame)
    
    # Display functions are commented out to avoid GUI errors.
    # cv2.imshow("Hand Points Overlay", frame)
    # if cv2.waitKey(1) & 0xFF == ord("q"):
    #     break
    
    frame_idx += 1

# ==============================
# Clean up
# ==============================
cap.release()
out.release()
# Comment out or remove the following line if GUI functions are not supported.
# cv2.destroyAllWindows()

print(f"Output video saved to: {output_video_path}")


Video size: 210x260, FPS: 1.0, Total frames: 20
Keypoints available for 116 frames.
Output video saved to: /nas/Chingiz/sing_language/code/imgTOvideo/imgTOvideo/01April_2010_Thursday_heute_default-2/20_1/hand_points_overlay.mp4


In [33]:
import cv2
import pickle
import numpy as np

# ==============================
# Configuration (adjust as needed)
# ==============================
video_path = '/nas/Chingiz/sing_language/code/imgTOvideo/imgTOvideo/01April_2010_Thursday_heute_default-2/20_1/original.mp4'
output_video_path = r'/nas/Chingiz/sing_language/code/imgTOvideo/imgTOvideo/01April_2010_Thursday_heute_default-2/20_1/hand_points_overlay_both.mp4'

# Path to the original and quantized keypoints pickle files
original_pkl_path  = r'/nas/Dataset/Phoenix/Chingiz/phoenix-2014-hands-keypoints.pkl'
quantized_pkl_path = r'/nas/Dataset/Phoenix/Chingiz/hoenix-2014-hands-quantized.pkl'

# Set the video key that exists in both pickle files.
video_key = "fullFrame-210x260px/train/01April_2010_Thursday_heute_default-2/1/01April_2010_Thursday_heute"

# Confidence threshold: only draw keypoints with confidence above this value.
CONF_THRESH = 0.5

# ==============================
# Load keypoints from both pickle files
# ==============================
with open(original_pkl_path, 'rb') as f:
    original_data = pickle.load(f)

with open(quantized_pkl_path, 'rb') as f:
    quantized_data = pickle.load(f)

if video_key not in original_data:
    raise ValueError(f"Video key '{video_key}' not found in the original pickle file.")
if video_key not in quantized_data:
    raise ValueError(f"Video key '{video_key}' not found in the quantized pickle file.")

# Extract keypoints. Expected shape: (n_frames, 42, 3)
orig_kp = original_data[video_key]['keypoints']
quant_kp = quantized_data[video_key]['keypoints']

# ==============================
# Open video file and set up VideoWriter
# ==============================
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Error opening video file!")

frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps          = cap.get(cv2.CAP_PROP_FPS)
frame_count  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video size: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {frame_count}")
print(f"Original keypoints available for {orig_kp.shape[0]} frames.")
print(f"Quantized keypoints available for {quant_kp.shape[0]} frames.")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# ==============================
# Process video frame by frame
# ==============================
frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Draw original keypoints (green filled circles)
    if frame_idx < orig_kp.shape[0]:
        kp_frame_orig = orig_kp[frame_idx]  # shape: (42, 3)
        for point in kp_frame_orig:
            x, y, conf = point
            if conf >= CONF_THRESH:
                cv2.circle(frame, (int(x), int(y)), radius=3, color=(0, 255, 0), thickness=-1)
    
    # Draw quantized keypoints (red circle outlines)
    if frame_idx < quant_kp.shape[0]:
        kp_frame_quant = quant_kp[frame_idx]  # shape: (42, 3)
        for point in kp_frame_quant:
            x, y, conf = point
            if conf >= CONF_THRESH:
                # thickness=2 (not filled) so that red circles are visible around original points.
                cv2.circle(frame, (int(x), int(y)), radius=3, color=(0, 0, 255), thickness=2)
    
    out.write(frame)
    
    # Since GUI display may not be supported, we comment these lines.
    # cv2.imshow("Hand Points Overlay (Original + Quantized)", frame)
    # if cv2.waitKey(1) & 0xFF == ord("q"):
    #     break
    
    frame_idx += 1

cap.release()
out.release()
# cv2.destroyAllWindows()  # Commented out to avoid GUI errors.

print(f"Output video saved to: {output_video_path}")


Video size: 210x260, FPS: 1.0, Total frames: 20
Original keypoints available for 116 frames.
Quantized keypoints available for 116 frames.
Output video saved to: /nas/Chingiz/sing_language/code/imgTOvideo/imgTOvideo/01April_2010_Thursday_heute_default-2/20_1/hand_points_overlay_both.mp4


In [36]:
import cv2
import pickle
import numpy as np

# ==============================
# Configuration (adjust as needed)
# ==============================
video_path = '/nas/Chingiz/sing_language/code/imgTOvideo/imgTOvideo/01April_2010_Thursday_heute_default-2/20_1/original.mp4'
output_video_path = r'/nas/Chingiz/sing_language/code/imgTOvideo/imgTOvideo/01April_2010_Thursday_heute_default-2/20_1/hand_points_overlay_trajectory_quan.mp4'
original_pkl_path  = r'/nas/Dataset/Phoenix/Chingiz/phoenix-2014-hands-keypoints.pkl'
quantized_pkl_path = r'/nas/Dataset/Phoenix/Chingiz/hoenix-2014-hands-quantized.pkl'

# Set the video key that exists in both pickle files.
video_key = "fullFrame-210x260px/train/01April_2010_Thursday_heute_default-2/1/01April_2010_Thursday_heute"

# Confidence threshold for drawing keypoints/trajectories.
CONF_THRESH = 0.5

# ==============================
# Load keypoints from both pickle files
# ==============================
with open(original_pkl_path, 'rb') as f:
    original_data = pickle.load(f)

with open(quantized_pkl_path, 'rb') as f:
    quantized_data = pickle.load(f)

if video_key not in original_data:
    raise ValueError(f"Video key '{video_key}' not found in the original pickle file.")
if video_key not in quantized_data:
    raise ValueError(f"Video key '{video_key}' not found in the quantized pickle file.")

# Extract keypoints.
# Expected shape: (n_frames, 42, 3)
orig_kp = original_data[video_key]['keypoints']
quant_kp = quantized_data[video_key]['keypoints']

# ==============================
# Open video file and set up VideoWriter
# ==============================
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Error opening video file!")

frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps          = cap.get(cv2.CAP_PROP_FPS)
frame_count  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video size: {frame_width}x{frame_height}, FPS: {fps}, Total frames: {frame_count}")
print(f"Original keypoints available for {orig_kp.shape[0]} frames.")
print(f"Quantized keypoints available for {quant_kp.shape[0]} frames.")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# ==============================
# Prepare trajectory storage
# ==============================
# For each of 42 joints, maintain a list of points for original and quantized keypoints.
orig_trajectories = {j: [] for j in range(42)}
quant_trajectories = {j: [] for j in range(42)}

# ==============================
# Process video frame by frame and draw trajectories
# ==============================
frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process original keypoints (green)
    if frame_idx < orig_kp.shape[0]:
        kp_frame_orig = orig_kp[frame_idx]  # shape: (42, 3)
        for j, point in enumerate(kp_frame_orig):
            x, y, conf = point
            if conf >= CONF_THRESH:
                pt = (int(x), int(y))
                orig_trajectories[j].append(pt)
                # Draw the trajectory as a polyline if there are at least 2 points.
                if len(orig_trajectories[j]) >= 2:
                    pts_array = np.array(orig_trajectories[j], np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [pts_array], isClosed=False, color=(0, 255, 0), thickness=1)
                # Draw current keypoint.
                cv2.circle(frame, pt, radius=3, color=(0, 255, 0), thickness=-1)
    
    # Process quantized keypoints (red)
    if frame_idx < quant_kp.shape[0]:
        kp_frame_quant = quant_kp[frame_idx]  # shape: (42, 3)
        for j, point in enumerate(kp_frame_quant):
            x, y, conf = point
            if conf >= CONF_THRESH:
                pt = (int(x), int(y))
                quant_trajectories[j].append(pt)
                # Draw the trajectory as a polyline if there are at least 2 points.
                if len(quant_trajectories[j]) >= 2:
                    pts_array = np.array(quant_trajectories[j], np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [pts_array], isClosed=False, color=(0, 0, 255), thickness=1)
                # Draw current keypoint.
                cv2.circle(frame, pt, radius=3, color=(0, 0, 255), thickness=2)
    
    out.write(frame)
    
    # Display is commented out (uncomment if running in an environment with GUI support)
    # cv2.imshow("Overlay with Trajectories", frame)
    # if cv2.waitKey(1) & 0xFF == ord("q"):
    #     break
    
    frame_idx += 1

cap.release()
out.release()
# cv2.destroyAllWindows()  # Commented out to avoid GUI errors.

print(f"Output video with trajectories saved to: {output_video_path}")


Video size: 210x260, FPS: 1.0, Total frames: 20
Original keypoints available for 116 frames.
Quantized keypoints available for 116 frames.
Output video with trajectories saved to: /nas/Chingiz/sing_language/code/imgTOvideo/imgTOvideo/01April_2010_Thursday_heute_default-2/20_1/hand_points_overlay_trajectory_quan.mp4
